In [24]:
import snscrape.modules.twitter as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, fbeta_score, confusion_matrix

In [ ]:
#belirlenen twitter haber sayfasından haber çekme işleminin gerçekleştirildiği fonksiyon :

In [25]:
def getNews(username,c_of_news):
    array_news = []
    for num_of_tw,tweet in enumerate(sns.TwitterSearchScraper("from:@"+username).get_items()):
        if num_of_tw>c_of_news:
            break
        regex = tweet.content.find("http")
        array_news.append(tweet.content[:regex])
    array_news = pd.DataFrame(array_news,columns=["news"])
    return array_news

In [ ]:
#verilere preprocessing işlemi uygulanarak; metin ve sayı dışındaki karakterleri silip,harflerin tamamını küçülten fonksiyon.

In [26]:
def prep(array_news):
    array_news["news"] = array_news["news"].apply(lambda x:" ".join(x.lower() for x in x.split()))
    array_news["news"] = array_news["news"].str.replace("[^\w\s]","")    
    array_news["news"] = array_news["news"].str.replace("[\d]","")
    return array_news

In [ ]:
#çekilen doğru ve yanlış haberler için, alt alta sıralanacak ve doğru haberler için 1 değeri, yanlış haberler için 0 değeri
#yazacak fonksiyon.

In [27]:
def join_news(true_news,wrong_news):
    true_news["value"] = 1
    wrong_news["value"] = 0
    news = pd.concat([true_news,wrong_news],axis=0,ignore_index=True)
    return news

In [ ]:
#verisetini logistic regression algoritmasına vererek öğreten ve gerçek ve tahmin değerlerini kullanarak istenilen değerleri
#hesaplayan fonksiyon.

In [28]:
def logistic_regression(traininput_ofdata,trainoutput_ofdata,testinput_ofdata,testoutput_ofdata):
    log_reg = LogisticRegression()
    log_reg_mdl = log_reg.fit(traininput_ofdata,trainoutput_ofdata)
    testdata_predict=log_reg_mdl.predict(testinput_ofdata)
    con_m=confusion_matrix(testoutput_ofdata, testdata_predict)
    pre_score=precision_score(testoutput_ofdata, testdata_predict)
    rec_score=recall_score(testoutput_ofdata, testdata_predict)
    f_score=fbeta_score(testoutput_ofdata, testdata_predict, beta=1)
    accuracy = cross_val_score(log_reg_mdl,testinput_ofdata,testoutput_ofdata,cv=10).mean()
    return accuracy,con_m,pre_score,rec_score,f_score,log_reg_mdl

In [ ]:
#tf-idf yöntemiyle veriler sayısal değerlere çevrilir ve bundan bir vektör oluşturuluyor.

In [29]:
def tf_idf(traininput_ofdata,testinput_ofdata):
    tf_idf_vectorizer = TfidfVectorizer()
    tf_idf_vectorizer.fit(traininput_ofdata)
    tf_idf_traininput = tf_idf_vectorizer.transform(traininput_ofdata)
    tf_idf_testinput = tf_idf_vectorizer.transform(testinput_ofdata)
    return tf_idf_traininput,tf_idf_testinput


In [ ]:
#verinin çekilip, kendini tekrarlamaması için kaydedilip onun kullanılması için gerekli kodlar.

In [30]:
#array_news=getNews("trthaber",1000)

In [ ]:
#array_news_wrong=getNews("zaytung",1000)

In [ ]:
#array_news=prep(array_news)
#array_news_wrong=prep(array_news_wrong)

In [ ]:
#news=join_news(array_news,array_news_wrong)

In [ ]:
#news.to_csv("haberler.csv",encoding = "utf-16",index=False)

In [31]:
news=pd.read_csv("haberler.csv",encoding="utf-16")

In [32]:
news.head()

,news,value
0,dünya genelinde koronavirüs tespit edilen kişi...,1
1,ton demiri at arabalarına yükleyerek çaldılar...,1
2,türkçenin derinliklerine dalınca gözlerime on ...,1
3,ak partide yılın ilk myk toplantısında gündem ...,1
4,gözler yılının aralık ayı enflasyon oranında ...,1


In [33]:
news.tail()

,news,value
1997,yargıda önemli reform tutukluluk kararı çıkmas...,0
1998,fotohaber türkiye merakla o soruşturmanın son...,0
1999,videohaber seydioğlu baklavaları tam diğer ba...,0
2000,görevden alınan merkez bankası başkanı murat ç...,0
2001,canon d için en iyi monteyi yapacak photoshop ...,0


In [ ]:
#veriler train ve test olarak ayrılır. Train:%70,Test=%30

In [34]:
traininput_ofdata,testinput_ofdata,trainoutput_ofdata,testoutput_ofdata=train_test_split(news["news"],news["value"],test_size=0.3)

In [ ]:
#sayısal değerleri katagorik değerlere çeviren kod.

In [35]:
encoder = preprocessing.LabelEncoder()
trainoutput_ofdata_2 = encoder.fit_transform(trainoutput_ofdata)
testoutput_ofdata_2 = encoder.fit_transform(testoutput_ofdata)

In [ ]:
#tf-idf fonksiyonu ile vektör oluşturuldu.

In [36]:
tf_idf_traininput,tf_idf_testinput = tf_idf(traininput_ofdata,testinput_ofdata)

In [37]:
tf_idf_traininput.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
tf_idf_testinput.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#logistic regression fonksiyonu ile istenilen değerler elde edildi.

In [39]:
accuracy,con_m,pre_score,rec_score,f_score,log_reg_mdl=logistic_regression(tf_idf_traininput,trainoutput_ofdata_2,tf_idf_testinput,testoutput_ofdata_2)

In [40]:
accuracy

0.883551912568306

In [41]:
con_m

array([[291,  23],
       [ 15, 272]], dtype=int64)

In [42]:
pre_score

0.9220338983050848

In [43]:
rec_score

0.9477351916376306

In [44]:
f_score

0.9347079037800688

In [ ]:
#farklı haberler çekilerek makineye tahmin etmesi için verilir.

In [45]:
new_news=getNews("Haberturk",10)

In [46]:
new_news

,news
0,Kadın kılığına girip 5 milyonluk cipi çaldı!
1,"Yargıtay, kocasına hakaret eden kadın ile doğu..."
2,"AK Parti'de Merkez Yürütme Kurulu (MYK) ""refor..."
3,#SONDAKİKA | Gabar'da öldürülen teröristlerden...
4,Pınar'ın katiliyle ilgili flaş tespit!
5,Fransa'da çatışma: 16 yaşında bir genç hayatın...
6,THY kargo uçağı zorunlu iniş yaptı
7,Gönüllere dokundu! Sokakta ameliyat
8,Feci yangın! Yaşlı adam hayatını kaybetti
9,İş Sanat Masal Tiyatrosu'nda bugün 'Rapunzel' ...


In [ ]:
#çekilen haberler düzenlendi, makine öğrenmesi algoritmasına bu haberler verildi ve tahminleri gözlendi.

In [47]:
new_preprocessing=prep(new_news)

In [48]:
new_preprocessing=pd.Series(new_preprocessing["news"])

In [49]:
vectorizer_2=TfidfVectorizer()
vectorizer_2.fit(traininput_ofdata)
new_news_vector = vectorizer_2.transform(new_preprocessing)

In [51]:
new_value=log_reg_mdl.predict(new_news_vector)

In [52]:
new_news["value"]=new_value

In [53]:
new_news

,news,value
0,kadın kılığına girip milyonluk cipi çaldı,1
1,yargıtay kocasına hakaret eden kadın ile doğum...,1
2,ak partide merkez yürütme kurulu myk reform gü...,0
3,sondakika gabarda öldürülen teröristlerden si...,1
4,pınarın katiliyle ilgili flaş tespit,1
5,fransada çatışma yaşında bir genç hayatını ka...,1
6,thy kargo uçağı zorunlu iniş yaptı,1
7,gönüllere dokundu sokakta ameliyat,1
8,feci yangın yaşlı adam hayatını kaybetti,1
9,iş sanat masal tiyatrosunda bugün rapunzel var...,1
